# 텍스트 벡터화: 사용자 정의 함수 활용

`TextVectorization` 레이어는 텍스트를 정수 시퀀스로 변환하는 편리한 도구입니다. 기본적으로 구두점 제거, 소문자 변환 등의 표준화와 공백 기준의 토큰화를 수행하지만, 사용자가 직접 함수를 정의하여 이 과정을 커스터마이징할 수 있습니다.

## 1. 라이브러리 임포트

In [ ]:
import re
import string
import tensorflow as tf
from keras.layers import TextVectorization

## 2. 사용자 정의 함수 정의

- **표준화(Standardization) 함수**: 텍스트를 전처리하는 단계입니다. 여기서는 텍스트를 소문자로 변환하고 정규식을 사용하여 모든 구두점을 제거합니다.
- **토큰화(Splitting) 함수**: 표준화된 텍스트를 단어(토큰) 단위로 분리하는 단계입니다. 여기서는 공백을 기준으로 분리합니다.

In [ ]:
# 사용자 정의 표준화 함수
def custom_standardization_fn(text):
    # 텍스트를 소문자로 변환
    lower_text = tf.strings.lower(text)
    # 정규식을 사용하여 구두점 제거
    return tf.strings.regex_replace(lower_text, f"[{re.escape(string.punctuation)}]", "")

# 사용자 정의 토큰 분리 함수
def custom_split_fn(text):
    # 공백을 기준으로 텍스트를 분리하여 토큰 리스트를 반환
    return tf.strings.split(text)

## 3. TextVectorization 레이어 초기화

`TextVectorization` 객체를 생성할 때 `standardize`와 `split` 매개변수에 위에서 정의한 사용자 정의 함수를 전달합니다.

In [ ]:
text_vectorization = TextVectorization(
    output_mode="int",                   # 출력 형식을 정수 시퀀스로 지정
    standardize=custom_standardization_fn, # 표준화에 사용할 함수 전달
    split=custom_split_fn                  # 토큰화에 사용할 함수 전달
)

## 4. 어휘 사전(Vocabulary) 생성

샘플 데이터셋을 `adapt` 메서드에 전달하여 텍스트 데이터에 있는 모든 고유한 단어(토큰)로 구성된 어휘 사전을 생성합니다. 이 사전은 단어를 정수 인덱스에 매핑하는 역할을 합니다.

In [ ]:
# 학습에 사용할 샘플 데이터셋
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms",
    "Dog is pretty"
]

# 데이터셋을 사용하여 어휘 사전을 생성
text_vectorization.adapt(dataset)

### 생성된 어휘 사전 확인

`get_vocabulary()` 메서드를 사용하여 생성된 사전을 확인할 수 있습니다. 단어들은 빈도수 순으로 정렬되며, 인덱스 0은 패딩(padding), 인덱스 1은 사전에 없는 단어(OOV, Out-of-Vocabulary)를 위해 예약되어 있습니다.

In [ ]:
vocabulary = text_vectorization.get_vocabulary()
print("어휘 사전:", vocabulary)

## 5. 인코딩 (텍스트 -> 정수 시퀀스)

어휘 사전이 생성되면, `TextVectorization` 레이어를 함수처럼 호출하여 새로운 텍스트를 정수 시퀀스로 변환(인코딩)할 수 있습니다.

In [ ]:
# 인코딩할 텍스트
text = "I write, rewrite, and still rewrite again"

# 텍스트를 정수 시퀀스로 변환
encoded = text_vectorization(text)
print("인코딩된 시퀀:", encoded)

## 6. 디코딩 (정수 시퀀스 -> 텍스트)

인코딩된 정수 시퀀스를 다시 사람이 읽을 수 있는 텍스트로 되돌리려면, 어휘 사전을 사용하여 인덱스를 단어로 매핑하는 딕셔너리를 만들어야 합니다.

In [ ]:
# 인덱스를 단어로 매핑하는 딕셔너리 생성
decoded_vocab = dict(enumerate(vocabulary))
print("인덱스-단어 맵:", decoded_vocab)

# 정수 시퀀스를 텍스트로 변환
decoded_sentence = " ".join(decoded_vocab[int(i)] for i in encoded)
print("\n디코딩된 문장:", decoded_sentence)